<a href="https://colab.research.google.com/github/zi-onion/ESAA/blob/main/YB_2%EC%A1%B0_%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B83.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 데이터전처리

In [ ]:
train=pd.read_csv("/content/drive/MyDrive/데이터/데이콘/영화관객예측/movies_train.csv")
test=pd.read_csv("/content/drive/MyDrive/데이터/데이콘/영화관객예측/movies_test.csv")
submission=pd.read_csv("/content/drive/MyDrive/데이터/데이콘/영화관객예측/submission.csv")

In [ ]:
print(' train 데이터 \n')
print(train.head(3))
print('\n test 데이터 \n')
print(test.head(3))
print('\n submission \n')
print(submission.head(3))

 train 데이터 

       title distributor genre release_time  time screening_rat director  \
0     개들의 전쟁    롯데엔터테인먼트    액션   2012-11-22    96      청소년 관람불가      조병옥   
1       내부자들      (주)쇼박스   느와르   2015-11-19   130      청소년 관람불가      우민호   
2  은밀하게 위대하게      (주)쇼박스    액션   2013-06-05   123       15세 관람가      장철수   

   dir_prev_bfnum  dir_prev_num  num_staff  num_actor  box_off_num  
0             NaN             0         91          2        23398  
1      1161602.50             2        387          3      7072501  
2       220775.25             4        343          4      6959083  

 test 데이터 

         title distributor   genre release_time  time screening_rat director  \
0       용서는 없다      시네마서비스     느와르   2010-01-07   125      청소년 관람불가      김형준   
1  아빠가 여자를 좋아해      (주)쇼박스  멜로/로맨스   2010-01-14   113       12세 관람가      이광재   
2          하모니   CJ 엔터테인먼트     드라마   2010-01-28   115       12세 관람가      강대규   

   dir_prev_bfnum  dir_prev_num  num_staff  num_actor  
0    3.005290e+0

In [ ]:
print('#### train info ####\n')
train.info()
print('\n#### test info ###\n')
test.info()

#### train info ####

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           600 non-null    object 
 1   distributor     600 non-null    object 
 2   genre           600 non-null    object 
 3   release_time    600 non-null    object 
 4   time            600 non-null    int64  
 5   screening_rat   600 non-null    object 
 6   director        600 non-null    object 
 7   dir_prev_bfnum  270 non-null    float64
 8   dir_prev_num    600 non-null    int64  
 9   num_staff       600 non-null    int64  
 10  num_actor       600 non-null    int64  
 11  box_off_num     600 non-null    int64  
dtypes: float64(1), int64(5), object(6)
memory usage: 56.4+ KB

#### test info ###

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243 entries, 0 to 242
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ----

**전략**

1) submission 형태에 title, box_off_num이 존재하므로 y_pred 결과 칼럼이 title, box_off_num 두개만 존재해야 한다.

2) train데이터와 test 데이터가 따로 주어졌으므로 train 데이터를 따로 train_test_split 하지 말고 이걸 이용해서 그대로 모델링하자

3) dir_prev_bfnum 칼럼의 경우 train과 test 데이터에서 결측치가 모두 많이 관측되었으므로 drop하자

4) release_time의 경우 datetime으로 나눠준 후 년/월/일로 각 칼럼으로 나누어 모두 int 형태를 가지도록 하자

5) genre의 고윳값을 살펴보고 괜찮을거같으면 원핫인코딩해주자

screening_rat 도 마찬가지

6) 트리기반 모델을 쓸것이므로 라벨인코딩해주자

7) director도 라벨인코딩 해주자

In [ ]:
# dir_prev_bfnum drop

train.drop(['dir_prev_bfnum'], axis=1, inplace=True)
test.drop(['dir_prev_bfnum'], axis=1, inplace=True)

In [ ]:
# release_time 년월일로 나누기

# train
train['release_time']= pd.to_datetime(train['release_time'])
# 새로운 칼럼으로 연도, 월, 일 분리
train['release_time_년'] = train['release_time'].dt.year
train['release_time_월'] = train['release_time'].dt.month
train['release_time_일'] = train['release_time'].dt.day
train.drop(['release_time'], axis=1, inplace=True)

# test
test['release_time']= pd.to_datetime(test['release_time'])
# 새로운 칼럼으로 연도, 월, 일 분리
test['release_time_년'] = test['release_time'].dt.year
test['release_time_월'] = test['release_time'].dt.month
test['release_time_일'] = test['release_time'].dt.day
test.drop(['release_time'], axis=1, inplace=True)

In [ ]:
# genre 고윳값 살펴보기
print('##train genre##\n')
print(train['genre'].value_counts())
print('##test genre##\n')
print(test['genre'].value_counts())

##train genre##

genre
드라마       221
다큐멘터리      93
멜로/로맨스     78
코미디        53
공포         42
액션         28
느와르        27
애니메이션      21
미스터리       17
SF         13
뮤지컬         5
서스펜스        2
Name: count, dtype: int64
##test genre##

genre
드라마       78
멜로/로맨스    39
다큐멘터리     33
코미디       23
액션        21
공포        19
느와르       12
애니메이션      8
미스터리       7
서스펜스       1
뮤지컬        1
SF         1
Name: count, dtype: int64


12개의 같은 장르 가지므로 원핫인코딩 해줄 수 있다.

In [ ]:
# genre 원핫인코딩
train = pd.get_dummies(train, columns=['genre'], dtype=int)
test = pd.get_dummies(test, columns=['genre'], dtype=int)

In [ ]:
# screening_rat 고윳값 살펴보기
print('##train ##\n')
print(train['screening_rat'].value_counts())
print('##test ##\n')
print(test['screening_rat'].value_counts())

##train ##

screening_rat
청소년 관람불가    204
15세 관람가     202
12세 관람가     102
전체 관람가       92
Name: count, dtype: int64
##test ##

screening_rat
청소년 관람불가    93
15세 관람가     73
12세 관람가     40
전체 관람가      37
Name: count, dtype: int64


같은 등급을 가지므로 원핫인코딩 해주자.

In [ ]:
# screening_rat 원핫인코딩
train = pd.get_dummies(train, columns=['screening_rat'], dtype=int)
test = pd.get_dummies(test, columns=['screening_rat'],dtype=int)

In [ ]:
# 배급사 분포 살펴보기
print('##train ##\n')
print(train['distributor'].value_counts())
print('##test ##\n')
print(test['distributor'].value_counts())

##train ##

distributor
CJ 엔터테인먼트        54
롯데엔터테인먼트         52
(주)NEW           30
(주)마운틴픽쳐스        29
(주)쇼박스           26
                 ..
OAL(올)            1
(주)에이원 엔터테인먼트     1
(주)콘텐츠 윙          1
위더스필름             1
퍼스트런              1
Name: count, Length: 169, dtype: int64
##test ##

distributor
CJ 엔터테인먼트          31
롯데엔터테인먼트           24
(주)마운틴픽쳐스          14
(주)쇼박스             12
골든타이드픽처스           12
                   ..
웃기씨네                1
오픈유컬쳐               1
CJ E&M Pictures     1
크리에이티브컴즈(주)         1
(주)씨타마운틴픽쳐스         1
Name: count, Length: 93, dtype: int64


length의 길이에 큰 차이를 보이고, 1개의 영화만 배급한 배급사 데이터가 꽤 있다.

length가 매우 많으므로 (169개) 원핫인코딩이 적절하지 않을 수 있다.

나는 트리기반 모델을 사용할 것이고, 트리기반 모델의 경우 숫자의 순서/ 크기를 고려하지 않기 때문에 라벨 인코딩을 진행하여도 괜찮다.

In [ ]:
# distributor 라벨 인코딩
from sklearn.preprocessing import LabelEncoder

label_encoder= LabelEncoder()
train['distributor_encoded']=label_encoder.fit_transform(train['distributor'])
train.drop(['distributor'], axis=1, inplace=True)
test['distributor_encoded']=label_encoder.fit_transform(test['distributor'])
test.drop(['distributor'], axis=1, inplace=True)

같은 이유로 감독 칼럼도 라벨인코딩 해주자

In [ ]:
# director 라벨 인코딩
train['director_encoded']=label_encoder.fit_transform(train['director'])
train.drop(['director'], axis=1, inplace=True)
test['director_encoded']=label_encoder.fit_transform(test['director'])
test.drop(['director'], axis=1, inplace=True)

In [ ]:
# 데이터 점검
print('## train data##\n')
print(train.head(3))
print('\n')
print(train.info())

print('\n\n ## test data##\n')
print(test.head(3))
print('\n')
print(test.info())

## train data##

       title  time  dir_prev_num  num_staff  num_actor  box_off_num  \
0     개들의 전쟁    96             0         91          2        23398   
1       내부자들   130             2        387          3      7072501   
2  은밀하게 위대하게   123             4        343          4      6959083   

   release_time_년  release_time_월  release_time_일  genre_SF  ...  genre_서스펜스  \
0            2012              11              22         0  ...           0   
1            2015              11              19         0  ...           0   
2            2013               6               5         0  ...           0   

   genre_애니메이션  genre_액션  genre_코미디  screening_rat_12세 관람가  \
0            0         1          0                      0   
1            0         0          0                      0   
2            0         1          0                      0   

   screening_rat_15세 관람가  screening_rat_전체 관람가  screening_rat_청소년 관람불가  \
0                      0                     0        

In [ ]:
# title 칼럼에 겹치는 값이 있는지 확인
print(train['title'].duplicated().any())
print(test['title'].duplicated().any())

False
False


title 칼럼을 제외하고는 모두 잘 전처리 된 것을 확인할 수 있다. title 값은 겹치는 값이 없으므로 인덱스와 동일하다고 생각해도 된다.



In [ ]:
# box_off_num을 맨 뒤로 재배치

train=train[['title', 'time', 'dir_prev_num', 'num_staff', 'num_actor', 'release_time_년', 'release_time_월', 'release_time_일',
       'genre_SF', 'genre_공포', 'genre_느와르', 'genre_다큐멘터리', 'genre_드라마',
       'genre_멜로/로맨스', 'genre_뮤지컬', 'genre_미스터리', 'genre_서스펜스', 'genre_애니메이션',
       'genre_액션', 'genre_코미디', 'screening_rat_12세 관람가',
       'screening_rat_15세 관람가', 'screening_rat_전체 관람가',
       'screening_rat_청소년 관람불가', 'distributor_encoded', 'director_encoded','box_off_num']]

In [ ]:
train.columns

Index(['title', 'time', 'dir_prev_num', 'num_staff', 'num_actor',
       'release_time_년', 'release_time_월', 'release_time_일', 'genre_SF',
       'genre_공포', 'genre_느와르', 'genre_다큐멘터리', 'genre_드라마', 'genre_멜로/로맨스',
       'genre_뮤지컬', 'genre_미스터리', 'genre_서스펜스', 'genre_애니메이션', 'genre_액션',
       'genre_코미디', 'screening_rat_12세 관람가', 'screening_rat_15세 관람가',
       'screening_rat_전체 관람가', 'screening_rat_청소년 관람불가', 'distributor_encoded',
       'director_encoded', 'box_off_num'],
      dtype='object')

In [ ]:
submission.head()

,title,box_off_num
0,용서는 없다,0
1,아빠가 여자를 좋아해,0
2,하모니,0
3,의형제,0
4,평행 이론,0


submission의 예시를 보면 title과 box_off_num 칼럼만 존재하므로 title 칼럼을 제외해서 모델링한 후 추후에 붙여주자.

# 모델링

In [ ]:
X_train= train.iloc[:,1:-1]
y_train= train['box_off_num']
X_test = test.iloc[:,1:]

## XGBoost

### train_test_split 없이

train데이터와 test 데이터의 갯수가 적으므로 train_test_split 없이 진행할 것이다. 그러나 이때 테스트 데이터에 실제 box_off_num 값이 없으므로 교차검증을 이용해야한다.

In [ ]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

xgb= XGBRegressor()
cv_scores = cross_val_score(xgb, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
rmse_scores = (-cv_scores) ** 0.5
print("Cross-Validation RMSE Scores:", rmse_scores)
print("Mean CV RMSE:", rmse_scores.mean())

xgb.fit(X_train, y_train)

y_pred= xgb.predict(X_test)

result = pd.DataFrame({
    'title': test['title'],
    'box_off_num': y_pred
})

Cross-Validation RMSE Scores: [1125387.17191532 1449398.52660608 1236587.42954236 1653953.25690901
 1910033.11925268]
Mean CV RMSE: 1475071.9008450913


**하이퍼 파라미터 튜닝**

In [ ]:


# 하이퍼파라미터 그리드 설정
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

# GridSearchCV 설정
grid_search = GridSearchCV(estimator=XGBRegressor(),
                           param_grid=param_grid,
                           scoring='neg_mean_squared_error',
                           cv=5,
                           verbose=1,
                           n_jobs=-1)

# 그리드 검색 수행
grid_search.fit(X_train, y_train)from sklearn.model_selection import GridSearchCV

# 최적의 하이퍼파라미터 출력
print("Best Hyperparameters:", grid_search.best_params_)


Fitting 5 folds for each of 243 candidates, totalling 1215 fits
Best Hyperparameters: {'colsample_bytree': 0.6, 'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 50, 'subsample': 1.0}


In [ ]:
xgb= XGBRegressor(colsample_bytree= 0.6, learning_rate= 0.1, max_depth= 4, n_estimators= 50, subsample= 1.0)
xgb.fit(X_train, y_train)

cv_scores = cross_val_score(xgb, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
rmse_scores = (-cv_scores) ** 0.5
print("Cross-Validation RMSE Scores:", rmse_scores)
print("Mean CV RMSE:", rmse_scores.mean())

y_pred= xgb.predict(X_test)

Cross-Validation RMSE Scores: [ 912830.24611155 1234835.2259489  1141235.4272426  1590956.65157882
 1615380.3651493 ]
Mean CV RMSE: 1299047.583206235


In [ ]:
# 최종 결과 데이터프레임 생성
result = pd.DataFrame({
    'title': test['title'],
    'box_off_num': y_pred
})

# 결과 확인
print(result.head())

result.to_csv('prediction_results1.csv', index=False)

         title  box_off_num
0       용서는 없다  1621497.000
1  아빠가 여자를 좋아해  1117071.125
2          하모니  1480766.500
3          의형제  2388607.250
4        평행 이론  1485974.375


**이유는 모르겠으나 제출하면 예상대로 train_test_split한게 가장 성능이 높다.**

### train_test_split 수행 결과

In [ ]:
from sklearn.model_selection import train_test_split

X_train_split, X_valid, y_train_split,y_valid= train_test_split(X_train,y_train,test_size=41, random_state=42)
xgb=XGBRegressor(colsample_bytree= 0.6, learning_rate= 0.1, max_depth= 4, n_estimators= 50, subsample= 1.0)
xgb.fit(X_train_split, y_train_split)

y_pred= xgb.predict(X_valid)
print('RMSE', np.sqrt(mean_squared_error(y_valid, y_pred)))

y_test_pred= xgb.predict(X_test)


RMSE 995096.2430619928


**가장 작은 RMSE가 출력되었다.**

In [ ]:
# 최종 결과 데이터프레임 생성
result = pd.DataFrame({
    'title': test['title'],
    'box_off_num': y_test_pred
})

# 결과 확인
print(result.head())

         title   box_off_num
0       용서는 없다  2.062677e+06
1  아빠가 여자를 좋아해  1.230452e+06
2          하모니  1.417644e+06
3          의형제  1.707728e+06
4        평행 이론  9.048149e+05


In [ ]:
result.to_csv('prediction_results.csv', index=False)

## LGBM

In [ ]:
from lightgbm import LGBMRegressor

lgbm= LGBMRegressor()
cv_scores = cross_val_score(lgbm, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
rmse_scores = (-cv_scores) ** 0.5
print("Cross-Validation RMSE Scores:", rmse_scores)
print("Mean CV RMSE:", rmse_scores.mean())

lgbm.fit(X_train, y_train)

y_pred= xgb.predict(X_test)

result = pd.DataFrame({
    'title': test['title'],
    'box_off_num': y_pred
})

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000780 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 542
[LightGBM] [Info] Number of data points in the train set: 480, number of used features: 20
[LightGBM] [Info] Start training from score 703754.108333
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

**하이퍼파라미터 튜닝**

In [ ]:
# 하이퍼파라미터 그리드 설정
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.05, 0.1],
    'num_leaves': [20, 31, 40],
    'min_child_samples': [10, 20, 30],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

# GridSearchCV 설정
grid_search = GridSearchCV(estimator=LGBMRegressor(),
                           param_grid=param_grid,
                           scoring='neg_mean_squared_error',
                           cv=5,
                           verbose=1,
                           n_jobs=-1)

# 그리드 검색 수행
grid_search.fit(X_train, y_train)

# 최적의 하이퍼파라미터 출력
print("Best Hyperparameters:", grid_search.best_params_)

Fitting 5 folds for each of 2187 candidates, totalling 10935 fits
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000173 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 636
[LightGBM] [Info] Number of data points in the train set: 600, number of used features: 21
[LightGBM] [Info] Start training from score 708181.751667
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

In [ ]:
lgbm= LGBMRegressor(colsample_bytree= 1.0, learning_rate= 0.05, max_depth= 3, min_child_samples= 20, n_estimators= 100, num_leaves=20, subsample= 0.6)
lgbm.fit(X_train, y_train)

cv_scores = cross_val_score(lgbm, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
rmse_scores = (-cv_scores) ** 0.5
print("Cross-Validation RMSE Scores:", rmse_scores)
print("Mean CV RMSE:", rmse_scores.mean())

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000075 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 636
[LightGBM] [Info] Number of data points in the train set: 600, number of used features: 21
[LightGBM] [Info] Start training from score 708181.751667
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

**XGB 성능이 더 좋다.**

## train_test_split 수행 결과

In [ ]:
from sklearn.model_selection import train_test_split

X_train_split, X_valid, y_train_split,y_valid= train_test_split(X_train,y_train,test_size=41, random_state=42)
lgbm=LGBMRegressor(colsample_bytree= 1.0, learning_rate= 0.05, max_depth= 3, min_child_samples= 20, n_estimators= 100, num_leaves=20, subsample= 0.6)
lgbm.fit(X_train_split, y_train_split)

y_pred= lgbm.predict(X_valid)
print('RMSE', np.sqrt(mean_squared_error(y_valid, y_pred)))

y_test_pred= lgbm.predict(X_test)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000057 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 604
[LightGBM] [Info] Number of data points in the train set: 559, number of used features: 21
[LightGBM] [Info] Start training from score 714564.985689
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

train_test_split 한 결과 성능이 개선되었지만 여전히 xgb가 더 좋다.